In [2]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from collections import defaultdict

In [ ]:
def skeletonize(img):
    """Alternative skeletonization implementation without ximgproc"""
    skel = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    while True:
        open_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
        temp = cv2.subtract(img, open_img)
        eroded = cv2.erode(img, element)
        skel = cv2.bitwise_or(skel, temp)
        img = eroded.copy()
        if cv2.countNonZero(img) == 0:
            break
    return skel

def process_fingerprint(image_path, output_path, target_size=None, upscale_factor=1.0):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    if target_size:
        image = cv2.resize(image, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif upscale_factor != 1.0:
        h, w = image.shape
        image = cv2.resize(image, (int(w*upscale_factor), int(h*upscale_factor)), 
                         interpolation=cv2.INTER_LANCZOS4)
    
    # 1. Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(image)
    
    # 2. Noise reduction
    denoised = cv2.bilateralFilter(enhanced, 9, 75, 75)
    
    # 3. Adaptive thresholding
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY_INV, 21, 7)
    
    # 4. Morphological operations
    kernel = np.ones((3,3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # 5. Skeletonization (using alternative method)
    skeleton = skeletonize(morph)
    
    # 6. Final inversion and saving
    result = cv2.bitwise_not(skeleton)
    cv2.imwrite(output_path, result)

def batch_process_fingerprints(input_folder, output_folder, upscale=False):
    """
    Process all fingerprints in a folder
    """
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.lower().endswith('.tif')]
    
    for filename in tqdm(files, desc="Processing Fingerprints"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        try:
            if upscale:
                process_fingerprint(input_path, output_path, upscale_factor=2.0)
            else:
                process_fingerprint(input_path, output_path)
        except Exception as e:
            print(f"\nError processing {filename}: {str(e)}")
            continue

if __name__ == "__main__":
    INPUT_FOLDER = "../data/raw"
    OUTPUT_FOLDER = "../data/processed"
    UPSCALE_IMAGES = True  # Set to True for 2x upscaling
    
    print("Starting fingerprint processing...")
    batch_process_fingerprints(INPUT_FOLDER, OUTPUT_FOLDER, upscale=UPSCALE_IMAGES)
    print("\nProcessing completed successfully!")

Starting fingerprint processing...


Processing Fingerprints: 100%|██████████| 2056/2056 [01:01<00:00, 33.22it/s]


Processing completed successfully!


In [3]:
random.seed(42) 

processed_dir  = "../data/processed"
train_dir  = "../data/final/train"
test_dir  = "../data/final/test"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

finger_groups = defaultdict(list)

for filename in sorted(os.listdir(processed_dir)):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg", ".tif")):
        continue
    parts = filename.split("_")
    if len(parts) < 3:
        continue
    class_id = f"{parts[0]}_{parts[1]}" 
    finger_groups[class_id].append(filename)

for class_id, files in finger_groups.items():
    files.sort(key=lambda f: int(f.split("_")[2].split(".")[0]))

    train_class_dir = os.path.join(train_dir, class_id)
    test_class_dir = os.path.join(test_dir, class_id)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    for i, file in enumerate(files):
        src = os.path.join(processed_dir, file)
        if i < 6:
            dst = os.path.join(train_class_dir, file)
        else:
            dst = os.path.join(test_class_dir, file)
        shutil.copy(src, dst)
        print(f"Copied {file} → {'train' if i < 6 else 'test'}/{class_id}")

print("Dataset split complete with subfolders as classes.")

Copied 001_1_1.tif → train/001_1
Copied 001_1_2.tif → train/001_1
Copied 001_1_3.tif → train/001_1
Copied 001_1_4.tif → train/001_1
Copied 001_1_5.tif → train/001_1
Copied 001_1_6.tif → train/001_1
Copied 001_1_7.tif → test/001_1
Copied 001_1_8.tif → test/001_1
Copied 001_2_1.tif → train/001_2
Copied 001_2_2.tif → train/001_2
Copied 001_2_3.tif → train/001_2
Copied 001_2_4.tif → train/001_2
Copied 001_2_5.tif → train/001_2
Copied 001_2_6.tif → train/001_2
Copied 001_2_7.tif → test/001_2
Copied 001_2_8.tif → test/001_2
Copied 001_3_1.tif → train/001_3
Copied 001_3_2.tif → train/001_3
Copied 001_3_3.tif → train/001_3
Copied 001_3_4.tif → train/001_3
Copied 001_3_5.tif → train/001_3
Copied 001_3_6.tif → train/001_3
Copied 001_3_7.tif → test/001_3
Copied 001_3_8.tif → test/001_3
Copied 001_4_1.tif → train/001_4
Copied 001_4_2.tif → train/001_4
Copied 001_4_3.tif → train/001_4
Copied 001_4_4.tif → train/001_4
Copied 001_4_5.tif → train/001_4
Copied 001_4_6.tif → train/001_4
Copied 001_4_7.t

In [6]:
test_dir = "../data/final/test/"

def apply_block_damage_smart(image, block_size=80, num_blocks=7):
    """Apply white block damage only on fingerprint area."""
    damaged = np.copy(image)
    height, width = image.shape
    mask = image < 250
    ys, xs = np.where(mask)

    if len(xs) == 0:
        return damaged

    for _ in range(num_blocks):
        idx = random.randint(0, len(xs) - 1)
        x_center, y_center = xs[idx], ys[idx]
        x1 = max(0, x_center - block_size // 2)
        y1 = max(0, y_center - block_size // 2)
        x2 = min(width, x1 + block_size)
        y2 = min(height, y1 + block_size)
        damaged[y1:y2, x1:x2] = 255
    return damaged

def apply_blur_damage(image, block_size=60, num_blocks=5):
    damaged = np.copy(image)
    height, width = image.shape

    for _ in range(num_blocks):
        x = random.randint(0, width - block_size)
        y = random.randint(0, height - block_size)
        roi = damaged[y:y+block_size, x:x+block_size]
        blurred = cv2.GaussianBlur(roi, (11, 11), 0)
        damaged[y:y+block_size, x:x+block_size] = blurred
    return damaged

def apply_elliptical_noise(image, num_ellipses=5):
    damaged = np.copy(image)
    height, width = image.shape

    for _ in range(num_ellipses):
        center = (
            random.randint(0, width),
            random.randint(0, height)
        )
        axes = (
            random.randint(20, 60), 
            random.randint(10, 30)   
        )
        angle = random.randint(0, 180)
        startAngle = 0
        endAngle = 360
        color = 255 
        thickness = -1 

        cv2.ellipse(damaged, center, axes, angle, startAngle, endAngle, color, thickness)
    return damaged


def apply_combined_damage(image):
    """Apply all three types of damage sequentially."""
    image = apply_block_damage_smart(image)
    image = apply_blur_damage(image)
    image = apply_elliptical_noise(image)
    return image

# test_images = []
# for root, _, files in os.walk(test_dir):
#     for file in files:
#         if file.endswith(".tif"):
#             test_images.append(os.path.join(root, file))
# print(f"Found {len(test_images)} test images to process")

# for image_path in tqdm(test_images, desc="Applying combined damage"):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

#     if image is not None:
#         damaged_image = apply_combined_damage(image)
#         cv2.imwrite(image_path, damaged_image)

# print(f"\nOverwritten {len(test_images)} images with combined damage in {test_dir}")

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import numpy as np
import os
from tqdm import tqdm
import cv2

train_dir = "../data/final/train"

def add_custom_noise(img_array):
    noise = np.random.normal(loc=0, scale=10, size=img_array.shape)
    noisy_img = img_array + noise
    return np.clip(noisy_img, 0, 255)

def add_motion_blur(img_array, degree=5, angle=0):
    image = img_array.copy()
    k = np.zeros((degree, degree))
    k[int((degree - 1) / 2), :] = np.ones(degree)
    M = cv2.getRotationMatrix2D((degree / 2 - 0.5, degree / 2 - 0.5), angle, 1)
    k = cv2.warpAffine(k, M, (degree, degree))
    k = k / np.sum(k)
    blurred = cv2.filter2D(image, -1, k)
    return blurred

def augment_and_save_images_in_place(input_dir, augmentations_per_image=5):
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=5,
        zoom_range=0.05,
        brightness_range=(0.9, 1.1),
        fill_mode='nearest'
    )

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        for fname in tqdm(os.listdir(class_path), desc=f"Augmenting {class_name}"):
            if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".tif")):
                continue

            fpath = os.path.join(class_path, fname)
            img = load_img(fpath, color_mode='grayscale')
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            prefix = os.path.splitext(fname)[0]
            i = 0
            for batch in datagen.flow(x, batch_size=1):
                aug_img_array = batch[0] 

                if np.random.rand() < 0.5:
                    aug_img_array = add_custom_noise(aug_img_array)

                if np.random.rand() < 0.3:
                    degree = np.random.randint(3, 7)
                    angle = np.random.uniform(-20, 20)
                    aug_img_array = add_motion_blur(aug_img_array, degree=degree, angle=angle)

                aug_img_array = np.clip(aug_img_array, 0, 255).astype('uint8')

                if aug_img_array.ndim == 2:
                    aug_img_array = np.expand_dims(aug_img_array, axis=-1)

                aug_img = array_to_img(aug_img_array)
                aug_img.save(os.path.join(class_path, f"{prefix}_aug{i+1}.png"))
                i += 1
                if i >= augmentations_per_image:
                    break

augment_and_save_images_in_place(train_dir, augmentations_per_image=5)
print("Augmentation done.")

Augmenting 031_257: 100%|██████████| 6/6 [00:00<00:00,  8.88it/s]

Augmentation done.


In [7]:
import os
import cv2
import random
from tqdm import tqdm

train_dir = "../data/final/train"
test_dir  = "../data/final/test"

print("\n🔧 Applying damage per class in training dataset...")

total_clean = 0
total_damaged = 0

for class_id in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_id)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.tif', '.png', '.jpg', '.jpeg', '.bmp'))]
    num_images = len(images)
    
    num_to_damage = int(num_images * 0.2)
    selected = random.sample(images, num_to_damage)

    for fname in selected:
        img_path = os.path.join(class_path, fname)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            damaged = apply_combined_damage(img)
            cv2.imwrite(img_path, damaged)

    total_clean += (num_images - num_to_damage)
    total_damaged += num_to_damage
    print(f"Class {class_id}: {num_images - num_to_damage} clean + {num_to_damage} damaged")

print(f"\nFinal training set: {total_clean + total_damaged} images total — {total_clean} clean + {total_damaged} damaged.")



🔧 Applying damage per class in training dataset...
Class 001_1: 29 clean + 7 damaged
Class 001_2: 29 clean + 7 damaged
Class 001_3: 29 clean + 7 damaged
Class 001_4: 29 clean + 7 damaged
Class 001_5: 29 clean + 7 damaged
Class 001_6: 29 clean + 7 damaged
Class 002_10: 29 clean + 7 damaged
Class 002_11: 29 clean + 7 damaged
Class 002_12: 29 clean + 7 damaged
Class 002_7: 29 clean + 7 damaged
Class 002_8: 29 clean + 7 damaged
Class 002_9: 29 clean + 7 damaged
Class 003_13: 29 clean + 7 damaged
Class 003_14: 29 clean + 7 damaged
Class 003_15: 29 clean + 7 damaged
Class 003_16: 29 clean + 7 damaged
Class 003_17: 29 clean + 7 damaged
Class 003_18: 29 clean + 7 damaged
Class 004_19: 29 clean + 7 damaged
Class 004_20: 29 clean + 7 damaged
Class 004_21: 29 clean + 7 damaged
Class 004_22: 29 clean + 7 damaged
Class 004_23: 29 clean + 7 damaged
Class 004_24: 29 clean + 7 damaged
Class 005_25: 29 clean + 7 damaged
Class 005_26: 29 clean + 7 damaged
Class 005_27: 29 clean + 7 damaged
Class 005_28